In [5]:
from skimage.feature import hog
from skimage.feature import local_binary_pattern
import cv2 
import os
import numpy as np
from tqdm import tqdm
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

In [6]:
train_pos = r"splitted-3\train\positif"
train_neg = r"splitted-3\train\negatif"

val_pos = r"splitted-3\val\positif"
val_neg = r"splitted-3\val\negatif"

test_pos = r"splitted-3\test\positif"
test_neg = r"splitted-3\test\negatif"

## Feature Extraction

In [7]:
def extract_feature_combined(pos_path, neg_path):
    # List to store extracted features of an image
    features = []
    
    # List to store class labels, 1 for live, 0 for spoof
    labels = []
    
    radius = 3
    # Number of neighbors to consider for LBP
    n_points = 8 * radius
    # Sampling type for LBP
    METHOD = 'uniform'
    
    path_array = [pos_path, neg_path]
    
    for path in path_array:
        # Storing all images in a folder in a list 'files'
        files = os.listdir(path)
        
        # Loop through the images in the folder with progress tracking
        for img in tqdm(files, desc=f"Processing images in {path}"):
            # Construct the full image path
            path_img = os.path.join(path, img)
            # Reading the image in grayscale using cv2
            img = cv2.imread(path_img, cv2.IMREAD_GRAYSCALE)
            
            # Resizing the image so all images are of same size
            resized_img = cv2.resize(img, (300, 300))
            
            # Extracting features using LBP
            lbp = local_binary_pattern(resized_img, n_points, radius, METHOD)
            lbp_features = lbp.flatten()
            
            # Extracting features using HOG
            hog_features, _ = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                                  cells_per_block=(2, 2), visualize=True)
            
            # Concatenating LBP and HOG features
            combined_features = np.concatenate((lbp_features, hog_features))
            
            class_identifier = 1 if 'pos' in path else 0
            
            features.append(combined_features)
            labels.append(class_identifier)
    
    return features, labels


In [8]:
# Training and testing datasets
com_x_trn,com_y_trn = extract_feature_combined(train_pos,train_neg)
com_x_val,com_y_val = extract_feature_combined(val_pos, val_neg)
com_x_tst,com_y_tst = extract_feature_combined(test_pos, test_neg)

Processing images in splitted-3\test\negatif: 100%|██████████| 69/69 [00:15<00:00,  4.60it/s]


In [9]:
type(com_y_trn), type(com_x_trn)

(list, list)

In [10]:
# print the shape of the training and testing datasets
print(f"Training dataset shape: {len(com_y_tst)}, with {com_y_trn.count(1)} positive and {com_y_trn.count(0)} negative samples")
print(f"Validation dataset shape: {len(com_x_val)}, with {com_y_val.count(1)} positive and {com_y_val.count(0)} negative samples")
print(f"Testing dataset shape: {len(com_x_tst)}, with {com_y_tst.count(1)} positive and {com_y_tst.count(0)} negative samples")

Training dataset shape: 123, with 247 positive and 322 negative samples
Validation dataset shape: 122, with 53 positive and 69 negative samples
Testing dataset shape: 123, with 54 positive and 69 negative samples


## Baseline SVM

### Train and Val Accuracy

In [11]:
# Create and fit the model
com_clf = svm.SVC()
com_clf.fit(com_x_trn,com_y_trn)

SVC()

In [12]:
# Predict on the test features, print the results
com_y_pred_val = com_clf.predict(com_x_val)
print("Combined Accuracy: "+str(accuracy_score(com_y_val, com_y_pred_val)))
print('\n')
print(classification_report(com_y_val, com_y_pred_val))

Combined Accuracy: 0.7868852459016393


              precision    recall  f1-score   support

           0       0.77      0.88      0.82        69
           1       0.81      0.66      0.73        53

    accuracy                           0.79       122
   macro avg       0.79      0.77      0.78       122
weighted avg       0.79      0.79      0.78       122



### Test Accuracy

In [13]:
com_y_pred_test = com_clf.predict(com_x_tst)
print("Combined Accuracy: "+str(accuracy_score(com_y_tst, com_y_pred_test)))
print('\n')
print(classification_report(com_y_tst, com_y_pred_test))

Combined Accuracy: 0.7804878048780488


              precision    recall  f1-score   support

           0       0.74      0.93      0.83        69
           1       0.86      0.59      0.70        54

    accuracy                           0.78       123
   macro avg       0.80      0.76      0.76       123
weighted avg       0.80      0.78      0.77       123



In [14]:
com_clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

## Hyperparameter Tuning

In [16]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 10],
              'gamma': [1, 0.1, 0.01],  
              'kernel': ['linear', 'rbf']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = True)

# fitting the model for grid search
grid.fit(com_x_trn, com_y_trn)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear', 'rbf']},
             verbose=True)

In [17]:
best_params = grid.best_params_
best_score = grid.best_score_
print(best_params)
print(best_score)

{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
0.6942710759198882


In [18]:
# predict with best parameters
svm_best = svm.SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'])
svm_best.fit(com_x_trn, com_y_trn)
grid_predictions = svm_best.predict(com_x_val)

print("Combined Accuracy: "+str(accuracy_score(com_y_val, grid_predictions)))
print('\n')
print(classification_report(com_y_val, grid_predictions))

Combined Accuracy: 0.7786885245901639


              precision    recall  f1-score   support

           0       0.81      0.80      0.80        69
           1       0.74      0.75      0.75        53

    accuracy                           0.78       122
   macro avg       0.77      0.78      0.78       122
weighted avg       0.78      0.78      0.78       122



In [19]:
# predict with best parameters
svm_best = svm.SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'])
svm_best.fit(com_x_trn, com_y_trn)
grid_predictions = svm_best.predict(com_x_tst)

print("Combined Accuracy: "+str(accuracy_score(com_y_tst, grid_predictions)))
print('\n')
print(classification_report(com_y_tst, grid_predictions))

Combined Accuracy: 0.7479674796747967


              precision    recall  f1-score   support

           0       0.74      0.84      0.79        69
           1       0.76      0.63      0.69        54

    accuracy                           0.75       123
   macro avg       0.75      0.74      0.74       123
weighted avg       0.75      0.75      0.74       123

